## balanced_profolio

In [65]:
import pandas as pd
import numpy as np
import math

In [66]:
path = '/Users/yutingmei/APlusBernstein-Project/data/pred/lstm_preds.csv'
target = pd.read_csv(path)

In [67]:
# target = target.iloc[1: , :]
# target = target.reset_index(drop=True)

In [68]:
target

,sc,mom,value,.pred_prob_sc_lstm,.pred_prob_mom_lstm,.pred_prob_value_lstm,.pred_sc_lstm,.pred_mom_lstm,.pred_value_lstm,s_&_p_500_ret,sc_1d_fwd_ret,mom_1d_fwd_ret,value_1d_fwd_ret
0,1,0,1,0.495916,0.551642,0.551642,0,1,0,NaN,0.005083,-0.000662,0.001758
1,0,0,1,0.507404,0.537511,0.537511,1,1,0,-0.000333,-0.008109,-0.002763,-0.001990
2,1,1,1,0.513883,0.548627,0.548627,1,1,0,-0.002512,0.008351,0.009081,0.006174
3,1,1,1,0.525089,0.551225,0.551225,1,1,0,0.006082,0.001307,0.007427,0.001928
4,0,1,1,0.535904,0.556041,0.556041,1,1,0,-0.000212,0.000261,0.001359,0.002828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,1,0,0,0.473095,0.508067,0.508067,0,1,0,-0.000215,0.001707,-0.004040,-0.002439
1842,0,0,0,0.475162,0.510649,0.510649,0,1,0,-0.000846,-0.003714,-0.004661,0.006674
1843,0,0,0,0.484604,0.517438,0.517438,0,1,0,0.006763,-0.013597,-0.007605,-0.009775
1844,1,0,1,0.492156,0.528105,0.528105,0,1,0,-0.007195,0.004891,-0.007206,0.008705


In [38]:
#path = '/Users/yutingmei/Projects/APlusBernstein-Project/data/raw/FactorTimingData-ABVU2022.xlsx'
#target = pd.read_excel(path, sheet_name = 'target')

In [5]:
#chceck this code I think there is something wrong with s&p data

# def balanced_profolio(target):
#     #target_ = target.iloc[:, 1:4]
#     # target = target.iloc[:, -3:]#the return values
#     start = 100000
#     #s_p = target.iloc[:, 4]
#     s_p = target.iloc[:, -4]#s&p
#     #s_p = target["s_&_p_500_ret"]
#     real = target.iloc[:, 0:3]#real values in 1 or 0
#     compare_result = []
#     pred = target.iloc[:,6:9]
#     op = target.iloc[:,-3:]
# #     long = []
# #     short = []
#     total_list = []
# #     direct = []
#     for i in range(target.shape[0]):
#         row = op.iloc[i, :]
#         if pred.iloc[i,:].sum() >= 2:
#             # pred direction
#             op_invest = 1
#             op_s_p = -1
#         else:
#             op_invest = -1
#             op_s_p = 1
#         add = sum(list(map(lambda x: (start / 3) * (1 + op_invest * x), row)))
#         total = (add) + ((1 + (op_s_p * s_p[i])) * start)
#         total_list.append(total)
#         compare_result.append((total - (start * 2)) / (start * 2))
#         start = total / 2 
#     return compare_result, total_list

In [6]:
# compare_result, total = balanced_profolio(target)

In [51]:
#pred = target.iloc[:, 6:9]
#balanced_profolio(target, pred = target.iloc[:, 6:9])

In [58]:
#understanding s&p trend 
#this is invest in s&p and let it go for the entire time period 
#let the s&p run for the entire trend our model should do better than that 
#here its previous start*s&p and then that value is multiplied to next s&p and so on

# s_p = target.iloc[:, 9]
#s_p = s_p.iloc[1: , :]

In [ ]:
# start = 100000
# value = 0
# l = []
# for i in s_p:
#     start = i*start
#     l.append(start)
#     start == start

In [69]:
def long_short_etf(target):
    start = 100000
    s_p = target.iloc[:, -4]#s&p
    #s_p = target["s_&_p_500_ret"]
    # real = target.iloc[:, 0:3]#real values in 1 or 0
    excess_return = []
    pred = target.iloc[:,6:9]
    op = target.iloc[:,-3:]
    # total_list = []
    return_list = []
    total_list = []
    for i in range(target.shape[0]):
        row = op.iloc[i, :]
        direct_pred = list(map(lambda x: -1 if x == 0 else 1, pred.iloc[i,:]))
        add = sum(list(map(lambda x,y: (start / 3) * (1 +  y * x), row, direct_pred)))
        # if pred.iloc[i,:].sum() >= 2:
        #     # pred direction
        #     op_invest = 1
        #     op_s_p = -1
        # else:
        #     op_invest = -1
        #     op_s_p = 1
        # add = sum(list(map(lambda x: (start / 3) * (1 + op_invest * x), row)))
        # total = (add) + ((1 + (op_s_p * s_p[i])) * start)
        # total_list.append(total)
        percentage = (add - start) / (start) 
        return_list.append(percentage)
        total_list.append(add)
        if i == target.shape[0] - 1:
            excess_return.append(math.nan)
        else:
            excess_return.append(percentage - s_p[i + 1])
        start = add 
    return excess_return, return_list, total_list

In [70]:
excess_return, return_list, total = long_short_etf(target)

In [71]:
return_list[:10]

[-0.0025007104558903664,
 -0.002960448947110505,
 0.0037526262584299187,
 0.0022685225142926027,
 -0.0004027571471250438,
 0.0028197954507109993,
 -0.006599531958871702,
 0.005073208418034815,
 0.0009221859645221845,
 0.002376905907637439]

In [72]:
excess_return[:10]

[-0.0021677373884987665,
 -0.00044866814912880495,
 -0.0023291464880024816,
 0.0024807235285046027,
 -0.0007510583623697438,
 0.0005131033233860991,
 0.005976660662588598,
 -0.005744733534471884,
 -0.004244002149982216,
 0.003724031498695439]

In [73]:
total[0]

99749.92895441096

In [74]:
total[-1]

88690.43295223032